In [23]:
import getpass
from pathlib import Path
import os
import pandas as pd
from tqdm import tqdm
import random
from sorting_algorithms import *
import time
import pickle
import shutil

In [24]:
user = getpass.getuser()

base = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT/"
src = base + "FullPreprocessed/"
indicator_src = base + "Indicators/"

nifti_paths = list(Path(src).glob('*.nii'))
indicator_paths = list(Path(indicator_src).glob('*.nii'))

keys = [os.path.basename(path) for path in nifti_paths]
indicator_keys = [os.path.basename(path) for path in indicator_paths]

r = random.Random(1)
r.shuffle(keys)

In [25]:
sublists = []
sublist_size = 100

for i in range(0, len(keys), sublist_size):
    sublists.append(keys[i : i+sublist_size])

In [26]:
def create_save(sort_alg, dst, final_name):
    
    file_name = str(int(time.time()))

    df = pd.DataFrame(
        columns=['result', 'diff_levels', 'time', 'session', 'user', 'undone', 'type'])

    df.to_csv(dst + "/" + file_name + ".csv", index=False)

    rel_path_to_save = "saves/" + file_name
    save_obj = {"sort_alg": sort_alg, "name": final_name,
                "image_directory": "", "path_to_save": rel_path_to_save,
                "user_directory_dict": {} , "scroll_allowed": False}

    f = open(dst + "/" + file_name + ".pickle", "wb")
    pickle.dump(save_obj, f)
    f.close()

def create_image_folder(sublist, src_dir, dst_dir, makedirs = True):
    
    if makedirs:
        os.makedirs(dst_dir + "/nifti", exist_ok=True)

    for img in sublist:
        src = src_dir + "/" + img
        dst = dst_dir + "/nifti/" + img
        shutil.copyfile(src, dst)


In [27]:
dst_dir = "/home/ORUNET/" + user + "/2303_Chalmers/Data/BWT_Annotation_Summer2023"

for i, sublist in tqdm(enumerate(sublists)):

    sublist_dir = f"{dst_dir}/{i + 1}"

    os.makedirs(sublist_dir, exist_ok=True)

    create_image_folder(sublist, src, sublist_dir)
    
    #Adds the indicator elements to all of the sublists
    create_image_folder(indicator_keys, indicator_src, sublist_dir, False)
    final_sublist = sublist + indicator_keys
    r.shuffle(final_sublist)
    
    sort_alg = HybridTrueSkill(final_sublist)

    create_save(sort_alg, sublist_dir, f"BWT Annotation {i + 1}")

    

    
    
    

75it [14:51, 11.89s/it]
